In [9]:
# Create or overwrite the .env file
with open(".env", "w") as f:
    f.write("NEWS_API_KEY=468c0aaedd694cdf905ed97c1dcdc0b4")


In [10]:
from dotenv import load_dotenv
import os

load_dotenv()  # Loads .env file from current directory

print("API KEY:", os.getenv("NEWS_API_KEY"))


API KEY: 468c0aaedd694cdf905ed97c1dcdc0b4


In [11]:
import os

# Create the 'data' folder if it doesn't already exist
os.makedirs("data", exist_ok=True)


In [17]:
import requests
import pandas as pd
import os
from datetime import datetime, timedelta
from dotenv import load_dotenv
from tqdm import tqdm

# Load API key from .env
load_dotenv()
API_KEY = os.getenv('NEWS_API_KEY')

# Function to fetch news
def fetch_news(query="stock market", from_date="2024-05-01", to_date="2024-05-07", page_size=100):
    url = "https://newsapi.org/v2/everything"
    all_articles = []

    # Only one page allowed on free plan
    params = {
        'q': query,
        'from': from_date,
        'to': to_date,
        'language': 'en',
        'sortBy': 'relevancy',
        'pageSize': page_size,
        'page': 1,  # Only one page
        'apiKey': API_KEY
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print('Failed to fetch:', response.json())
        return pd.DataFrame()

    data = response.json()
    articles = data.get('articles', [])
    all_articles.extend(articles)

    return pd.DataFrame(all_articles)
   

In [18]:
def fetch_news(query_list, from_date='2024-05-01', to_date='2024-05-07', page_size=100):
    url = 'https://newsapi.org/v2/everything'
    all_articles = []
    
    for query in query_list:
        print(f"Fetching news for query: {query}")
        
        params = {
            'q': query,
            'from': from_date,
            'to': to_date,
            'language': 'en',
            'sortBy': 'relevancy',
            'pageSize': page_size,
            'page': 1,  # Only one page for each query
            'apiKey': API_KEY
        }
        
        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f'Failed to fetch for query: {query}, response: {response.json()}')
            continue  # Skip this query if it fails

        data = response.json()
        articles = data.get('articles', [])
        all_articles.extend(articles)
    
    return pd.DataFrame(all_articles)


In [14]:
query_list = ["stock market", "inflation", "NASDAQ", "financial news", "economy"]
df_news = fetch_news(query_list, from_date=str(week_ago), to_date=str(today))

if not df_news.empty:
    df_news = df_news[["publishedAt", "title", "description", "url", "source"]]
    df_news.to_csv("data/financial_news.csv", index=False)
    df_news.head()
else:
    print("No articles fetched.")


NameError: name 'week_ago' is not defined

In [15]:
import os

# Create the 'data' directory if it doesn't exist
os.makedirs("data", exist_ok=True)

# Save the data into a CSV file
df_news.to_csv("data/financial_news.csv", index=False)

# Check the first few rows of the dataframe
df_news.head()


NameError: name 'df_news' is not defined

In [16]:
!pip install psaw


In [ ]:
env_content = """REDDIT_CLIENT_ID=mNM6bsmfJfWlE24n1skAOg
REDDIT_CLIENT_SECRET=VldZScwuDTfLYsGpEw1MpRD9-t-lBQ
REDDIT_USER_AGENT=financial_news_script
"""

with open(".env", "w") as f:
    f.write(env_content)

print(".env file created successfully!")


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

print("Client ID:", os.getenv("REDDIT_CLIENT_ID"))


In [ ]:
!pip install praw


In [ ]:
import praw
import os
from dotenv import load_dotenv


In [ ]:
import praw
import os
from dotenv import load_dotenv
import pandas as pd
from datetime import datetime

load_dotenv()

reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT")
)

subreddits = ["StockMarket", "investing", "wallstreetbets"]
posts = []

for sub in subreddits:
    for post in reddit.subreddit(sub).top(time_filter='week', limit=100):
        posts.append({
            'subreddit': sub,
            'title': post.title,
            'description': post.selftext,
            'url': post.url,
            'publishedAt': datetime.utcfromtimestamp(post.created_utc),
            'source': 'Reddit'
        })

df_reddit = pd.DataFrame(posts)
df_reddit.to_csv("data/reddit_financial_news.csv", index=False)
df_reddit.head()


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def scrape_yahoo_finance_news():
    url = "https://finance.yahoo.com"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    headlines = []
    for item in soup.select("li.js-stream-content"):
        title_tag = item.find("h3")
        link_tag = item.find("a")
        if title_tag and link_tag:
            headlines.append({
                "title": title_tag.text.strip(),
                "url": "https://finance.yahoo.com" + link_tag["href"],
                "publishedAt": datetime.now(),
                "description": "",
                "source": "Yahoo Finance"
            })

    return pd.DataFrame(headlines)

# Run it
df_yahoo = scrape_yahoo_finance_news()
df_yahoo.to_csv("data/yahoo_financial_news.csv", index=False)
df_yahoo.head()


In [ ]:
!pip install feedparser


In [ ]:
import feedparser
import pandas as pd
from datetime import datetime

def fetch_yahoo_finance_rss():
    rss_url = "https://finance.yahoo.com/news/rssindex"
    feed = feedparser.parse(rss_url)

    news_items = []
    for entry in feed.entries:
        news_items.append({
            "title": entry.title,
            "url": entry.link,
            "publishedAt": datetime(*entry.published_parsed[:6]),
            "description": entry.get("summary", ""),
            "source": "Yahoo Finance"
        })

    return pd.DataFrame(news_items)

# Run and save
df_yahoo_rss = fetch_yahoo_finance_rss()
df_yahoo_rss.to_csv("data/yahoo_financial_news.csv", index=False)
df_yahoo_rss.head()


In [ ]:
import pandas as pd

# Load datasets
df_reddit = pd.read_csv("data/reddit_financial_news.csv")
df_yahoo = pd.read_csv("data/yahoo_financial_news.csv")

# Ensure consistent column names
common_cols = ["title", "url", "publishedAt", "description", "source"]
df_reddit = df_reddit[common_cols]
df_yahoo = df_yahoo[common_cols]

# Merge datasets
df_all = pd.concat([df_reddit, df_yahoo], ignore_index=True)

# Drop duplicates (if any)
df_all.drop_duplicates(subset=["title", "url"], inplace=True)

# Save merged file
df_all.to_csv("data/all_financial_news.csv", index=False)

df_all.head()


In [ ]:
# Load the financial_news.csv dataset (if available)
df_financial_news = pd.read_csv("data/financial_news.csv")

# Ensure the column names are consistent across all datasets
df_financial_news = df_financial_news[common_cols]

# Merge everything into one final dataset
df_all = pd.concat([df_reddit, df_yahoo, df_financial_news], ignore_index=True)

# Drop duplicates based on title and URL
df_all.drop_duplicates(subset=["title", "url"], inplace=True)

# Save the final merged dataset
df_all.to_csv("data/all_financial_news.csv", index=False)

df_all.head()


In [ ]:
!pip install nltk


In [ ]:
import nltk
nltk.download('vader_lexicon')


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Combine title + description for context
df_all["text"] = df_all["title"].fillna("") + ". " + df_all["description"].fillna("")

# Initialize analyzer
sid = SentimentIntensityAnalyzer()

# Apply sentiment scoring
df_all["sentiment_score"] = df_all["text"].apply(lambda x: sid.polarity_scores(x)["compound"])
df_all["sentiment_label"] = df_all["sentiment_score"].apply(
    lambda score: "positive" if score > 0.05 else ("negative" if score < -0.05 else "neutral")
)

# Save results
df_all.to_csv("data/news_with_sentiment.csv", index=False)
df_all[["publishedAt", "source", "title", "sentiment_label"]].head(10)


In [ ]:
# Automatically infer datetime format and handle errors
df_all['publishedAt'] = pd.to_datetime(df_all['publishedAt'], errors='coerce')

# Check the conversion
df_all['publishedAt'].head()


In [ ]:
# Automatically infer datetime format and handle errors
df_all['publishedAt'] = pd.to_datetime(df_all['publishedAt'], errors='coerce')

# Check the conversion
df_all['publishedAt'].head()


In [ ]:
# Remove 'Z' (UTC timezone) if present and convert to datetime
df_all['publishedAt'] = df_all['publishedAt'].str.replace('Z', '', regex=False)
df_all['publishedAt'] = pd.to_datetime(df_all['publishedAt'], errors='coerce')

# Check the conversion
df_all['publishedAt'].head()


In [ ]:
import pandas as pd

df_all = pd.read_csv("data/all_financial_news.csv")

# Fix datetime parsing by allowing mixed or ISO formats
df_all['publishedAt'] = pd.to_datetime(df_all['publishedAt'], errors='coerce', format='ISO8601')

# Drop rows where datetime conversion failed
df_all.dropna(subset=['publishedAt'], inplace=True)

# Sort values
df_all.sort_values(by='publishedAt', ascending=True, inplace=True)

df_all['publishedAt'].head()


In [ ]:
import pandas as pd

# Load data
df_all = pd.read_csv("data/all_financial_news.csv")

# Convert to datetime in UTC to make all timestamps tz-aware
df_all['publishedAt'] = pd.to_datetime(df_all['publishedAt'], errors='coerce', utc=True)

# Drop rows with invalid dates
df_all.dropna(subset=['publishedAt'], inplace=True)

# Sort chronologically
df_all.sort_values(by='publishedAt', ascending=True, inplace=True)

# Preview
df_all['publishedAt'].head()


In [ ]:
!pip install vaderSentiment

import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


In [ ]:
# Ensure datetime format (force conversion)
df_all['publishedAt'] = pd.to_datetime(df_all['publishedAt'], errors='coerce', utc=True)

# Drop rows where datetime conversion failed
df_all.dropna(subset=['publishedAt'], inplace=True)

# Now extract the date
df_all['date'] = df_all['publishedAt'].dt.date


In [ ]:
# Group by date to get daily average sentiment
daily_sentiment = df_all.groupby('date')['sentiment_score'].mean().reset_index()

# Add risk alert flag based on threshold
risk_threshold = -0.3
daily_sentiment['risk_alert'] = daily_sentiment['sentiment_score'] < risk_threshold

# Display risky days
daily_sentiment[daily_sentiment['risk_alert']]


In [ ]:
# Define risk threshold
risk_threshold = -0.3

# Add risk alert flag
daily_sentiment['risk_alert'] = daily_sentiment['sentiment_score'] < risk_threshold

# Show risky days
risky_days = daily_sentiment[daily_sentiment['risk_alert'] == True]
risky_days


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))
plt.plot(daily_sentiment['date'], daily_sentiment['sentiment_score'], marker='o')
plt.axhline(y=risk_threshold, color='red', linestyle='--', label='Risk Threshold')
plt.title("Daily Average Financial Sentiment")
plt.xlabel("Date")
plt.ylabel("Sentiment Score")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Step 1: Create a 'date' column (drop the time part)
df_all['date'] = df_all['publishedAt'].dt.date

# Step 2: Aggregate sentiment per day (mean sentiment)
daily_sentiment = df_all.groupby('date')['sentiment_score'].mean().reset_index()

# Step 3: Calculate a 3-day rolling average
daily_sentiment['rolling_sentiment'] = daily_sentiment['sentiment_score'].rolling(window=3).mean()

# Step 4: Define risk levels based on thresholds
def classify_risk(score):
    if score < -0.2:
        return '🔴 High Risk'
    elif score < 0:
        return '🟠 Medium Risk'
    else:
        return '🟢 Low Risk'

daily_sentiment['risk_level'] = daily_sentiment['rolling_sentiment'].apply(classify_risk)

# Show the results
daily_sentiment.tail(10)


In [8]:
# Make sure 'daily_sentiment' has 'sentiment_score' and 'date'
print(daily_sentiment.columns)

# Define a risk threshold
risk_threshold = -0.2

# Create the 'risk_alert' column safely
if 'sentiment_score' in daily_sentiment.columns:
    daily_sentiment['risk_alert'] = daily_sentiment['sentiment_score'] < risk_threshold
else:
    raise ValueError("Column 'sentiment_score' not found in daily_sentiment.")


NameError: name 'daily_sentiment' is not defined

In [ ]:
import matplotlib.pyplot as plt

# Plot daily sentiment
plt.figure(figsize=(14, 6))
plt.plot(daily_sentiment['date'], daily_sentiment['sentiment_score'], marker='o', label='Sentiment Score')

# Highlight risk alert days
risk_days = daily_sentiment[daily_sentiment['risk_alert']]
plt.scatter(risk_days['date'], risk_days['sentiment_score'], color='red', label='Risk Alert', zorder=5)

# Plot formatting
plt.axhline(y=risk_threshold, color='gray', linestyle='--', label='Risk Threshold')
plt.title('Daily Financial News Sentiment with Risk Alerts')
plt.xlabel('Date')
plt.ylabel('Average Sentiment Score')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.xticks(rotation=45)
plt.show()


In [19]:
import pandas as pd
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sandi\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True